In [1]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import time
import requests

In [2]:
login = 121391564
password = 'xxxxxxxxx'
server = 'Exness-MT5Trial7'

In [3]:
# function to send a market order
def market_order(symbol, volume, order_type, **kwargs):
    tick = mt5.symbol_info_tick(symbol)

    order_dict = {'buy' : 0, 'sell' : 1}
    price_dict = {'buy' : tick.ask, 'sell' : tick.bid}

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": order_dict[order_type],
        "price": price_dict[order_type],
        "deviation": DEVIATION,
        "magic": 22085,
        "comment": "python market order",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    order_result = mt5.order_send(request)
    print(order_result)
    return order_result

In [4]:
# function to close an order base on the ticket id
def close_order(ticketID):
    positionIDs = mt5.positions_get()

    for pos in positionIDs:
        tick = mt5.symbol_info_tick(pos.symbol) #current price
        type_dict = {0:1, 1:0} # 0 represents buy, 1 represents sell - inverting order_type to close the position
        price_dict = {0 : tick.ask, 1 : tick.bid}

        if pos.ticket == ticketID:
            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "position": pos.ticket,
                "symbol": pos.symbol,
                "volume": pos.volume,
                "type": type_dict[pos.type],
                "price": price_dict[pos.type],
                "deviation": DEVIATION,
                "magic": 22085,
                "comment": "python close order",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }

            order_result = mt5.order_send(request)
            print(order_result)
            return order_result
    return "TicketID does not exist."

In [5]:
# function to get the exposure of a symbol
def get_exposure(symbol):
    positions = mt5.positions_get(symbol = symbol)
    if positions :
        pos_df = pd.DataFrame(positions, columns=positions[0]._asdict().keys())
        exposure = pos_df['volume'].sum()
        return exposure

In [6]:
# function to look for trading signals, what trend is now?
def signal(symbol, timeframe, sma_period) :
    bars = mt5.copy_rates_from_pos(symbol, timeframe, 1, sma_period)
    bars_df = pd.DataFrame(bars)

    last_close = bars_df.iloc[-1].close
    sma = round(bars_df.close.mean(),2)

    direction = 'flat'
    if last_close > sma:
        direction = 'buy'
    elif last_close < sma:
        direction = 'sell'
    
    return last_close, sma, direction

In [7]:
if __name__ == '__main__' :

    #PARAMETERS -----------
    SYMBOL = "BTCUSD"               #HERE <--------------------------
    VOLUME = 0.01                   #HERE <--------------------------
    TIMEFRAME = mt5.TIMEFRAME_M5    #HERE <--------------------------
    SMA_PERIOD = 100
    DEVIATION = 100

    #Connect and login
    mt5.initialize()
    
    mt5.login(login, password, server)

    print('Account :', mt5.account_info().login)
    print('Balance :', mt5.account_info().balance)
    print('Equity :', mt5.account_info().equity)
    print('Exposure :', get_exposure(SYMBOL))

    #chatbot
    token = "5868960988:AAHXC_mJbjt4EA_aknIBBp5NK6ddLb9HDU0"
    url = f"https://api.telegram.org/bot{token}"
    
    while True :

        while datetime.now().minute%5 == 0:     #HERE <--------------------------
            exposure = get_exposure(SYMBOL)

            # calculating last candle close and simple moving average and checking for trading signal
            last_close, sma, direction = signal(SYMBOL, TIMEFRAME, SMA_PERIOD)

            if direction == 'buy':
                # if we have a BUY signal, close all short positions
                for pos in mt5.positions_get():
                    if pos.type == 1:
                        close_order(pos.ticket)
                        params = {"chat_id": "5297105149", "text": f"Close Sell, Balance : {mt5.account_info().balance}"}
                        r = requests.get(url + "/sendMessage", params=params)
                
                # if there are no open positions, open a new long position
                if not mt5.positions_total():
                    market_order(SYMBOL, VOLUME, direction)
                    params = {"chat_id": "5297105149", "text": f"Open Buy, Balance : {mt5.account_info().balance}"}
                    r = requests.get(url + "/sendMessage", params=params)
            
            elif direction == 'sell':
                # if we have a SELL signal, close all short positions
                for pos in mt5.positions_get():
                    if pos.type == 0:
                        close_order(pos.ticket)
                        params = {"chat_id": "5297105149", "text": f"Close Buy, Balance : {mt5.account_info().balance}"}
                        r = requests.get(url + "/sendMessage", params=params)

                # if there are no open positions, open a new short position
                if not mt5.positions_total():
                    market_order(SYMBOL, VOLUME, direction)
                    params = {"chat_id": "5297105149", "text": f"Open Sell, Balance : {mt5.account_info().balance}"}
                    r = requests.get(url + "/sendMessage", params=params)

            print('time: ', datetime.now())
            print('exposure: ', exposure)
            print('last_close: ', last_close)
            print('sma: ', sma)
            print('signal: ', direction)
            print('-------\n')

            # update every 30 second
            time.sleep(30)

Account : 121391564
Balance : 1301.83
Equity : 1301.83
Exposure : None
OrderSendResult(retcode=10009, deal=73404240, order=137328718, volume=0.01, price=25828.55, bid=25828.55, ask=25835.510000000002, comment='python market or', request_id=357398476, retcode_external=0, request=TradeRequest(action=1, magic=22085, order=0, symbol='BTCUSD', volume=0.01, price=25828.55, stoplimit=0.0, sl=0.0, tp=0.0, deviation=100, type=1, type_filling=1, type_time=0, expiration=0, comment='python market order', position=0, position_by=0))
time:  2023-06-12 22:30:39.211631
exposure:  None
last_close:  25821.86
sma:  25921.09
signal:  sell
-------



KeyboardInterrupt: 